In [1]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS
from od.env.station import BS_PRESET
import od.misc.statistic as oms

In [2]:
print(os.getcwd())
statistics = []
for i,loc in enumerate([ResourceAllocatorType.NOMA_OPT,ResourceAllocatorType.NOMA_APR]):
    interest_config = InterestConfig(True,loc,True,1.3,7)
    with open("../data/GS20/{}/object.pickle".format(interest_config.folder()), "rb") as file:
        statistics.append(pickle.load(file))

/home/OD/repos/ProjectSumo/ipynb


In [3]:
statistics

In [4]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV
def RecordPrinter(header):
    print("time at:{}".format(header.at))
    print("bs ot:{}".format(header.is_bs_ot))
    print("src ot:{}".format(header.is_src_ot))
    print("time bs drop:{}".format(header.time_bs_drop))
    print("time bs serv:{}".format(header.time_bs_serv))
    print("time bs tx:{}".format(header.time_bs_tx))
    print("time bs txq:{}".format(header.time_bs_txq))
    print("time bs trip:{}".format(header.time_veh_trip))

In [5]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2, UMI-1, UMI-2, UMI-3, UMI-4, UMI-5]

In [6]:
for statistic in statistics:
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            for bs in GV.NET_STATION_CONTROLLER:
                if(len(record.time_bs_txq[bs])==0):
                    continue
                if(record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS):
                    record.is_bs_ot[bs] = True

In [7]:
for i,statistic in enumerate(statistics):
    print("====={}======".format(i))
    for sg in statistic.social_group:
        ot_count = 0 
        for header, record in statistic.sg_header[sg].items():
            if record.is_src_ot > 0:
                ot_count +=1
            elif (True in [record.is_bs_ot[i] for i in GV.NET_STATION_CONTROLLER]):
                ot_count +=1

        print("{}: {} => {}".format(sg,ot_count, ot_count / max(len(statistic.sg_header[sg]),1)))
            

=====0======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 122 => 0.6455026455026455
=====1======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 1133 => 0.7139256458727158
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 189 => 1.0


=====0======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 122 => 0.6455026455026455
=====1======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 122 => 0.6455026455026455


In [9]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [10]:
target_header = "flow_2.34-117"
# target_header = "flow_0.58-99"
for statistic in statistics:
    header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
    print("In {}".format(header_in_sg_list))
    print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
    print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
    print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In ['CRASH', 'RCWS']
C?:True
G?:True
C2G?:True
In ['CRASH', 'RCWS']
C?:True
G?:True
C2G?:True


In [11]:
for i,statistic in enumerate(statistics):
    for sg in statistic.social_group:
        if target_header in statistic.sg_header[sg]:
            header = statistic.sg_header[sg][target_header]
            print("======={}:{}========".format(i,sg))
            RecordPrinter(header)

=======0:CRASH========
time at:186.439
bs ot:[False, False, False, False, False, False, False]
src ot:False
time bs drop:[-1, -1, -1, -1, -1, -1, -1]
time bs serv:[-1, -1, -1, 186.444, -1, -1, -1]
time bs tx:[[], [], [], [[186.4435, 186.444], [186.444, 186.4445]], [], [], []]
time bs txq:[[], [], [], [[186.443, 186.444], [186.444, 186.444]], [], [], []]
time bs trip:{'flow_0.37': 0.005500000000012051, 'flow_0.38': 0.005500000000012051, 'flow_0.39': 0.005500000000012051, 'flow_0.40': 0.005500000000012051, 'flow_0.41': 0.005500000000012051, 'flow_0.42': 0.005500000000012051, 'flow_0.43': 0.005500000000012051, 'flow_0.44': 0.005500000000012051, 'flow_0.45': 0.005500000000012051, 'flow_0.46': 0.005500000000012051, 'flow_0.47': 0.005500000000012051, 'flow_0.48': 0.005500000000012051, 'flow_1.7': 0.005500000000012051, 'flow_1.8': 0.005500000000012051, 'flow_13.3': 0.005500000000012051, 'flow_13.4': 0.005500000000012051, 'flow_2.28': 0.005500000000012051, 'flow_2.29': 0.005500000000012051, 'f

In [23]:
bs_index = 1
social_groups = statistics[0].social_group
print(social_groups)
app_stats_fn = lambda x: statistic.sg_header[social_groups[-1]]
# app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

[<CRASH(QoS:0, Gid:0)>, <RCWS(QoS:1, Gid:1)>, <DYN_SG_0(QoS:1, Gid:2)>, <DYN_SG_1(QoS:1, Gid:3)>, <DYN_SG_2(QoS:1, Gid:4)>, <DYN_SG_3(QoS:1, Gid:5)>, <DYN_SG_4(QoS:1, Gid:6)>, <DYN_SG_5(QoS:1, Gid:7)>, <DYN_SG_6(QoS:1, Gid:8)>, <DYN_SG_7(QoS:1, Gid:9)>, <DYN_SG_8(QoS:1, Gid:10)>]


In [24]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x : x.time_gen*1000000+x.time_serv)
    for helper in sorted(bs_serv_list[i],key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits):
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [25]:
for v in bs_serv_list[0]:
    print(v)

flow_2.79-1[ at:186.916 ,to:193.678, dur:6.762, size:3416]
flow_2.79-3[ at:186.933 ,to:193.678, dur:6.745, size:13112]
flow_2.79-5[ at:186.95 ,to:193.678, dur:6.728, size:8112]
flow_2.79-7[ at:186.966 ,to:193.686, dur:6.72, size:15856]
flow_2.79-9[ at:186.983 ,to:193.688, dur:6.705, size:14936]
flow_2.79-11[ at:186.999 ,to:193.688, dur:6.689, size:7912]
flow_2.79-12[ at:187.015 ,to:193.696, dur:6.681, size:10248]
flow_2.79-14[ at:187.031 ,to:193.696, dur:6.665, size:10792]
flow_2.79-16[ at:187.047 ,to:193.696, dur:6.649, size:928]
flow_2.79-18[ at:187.063 ,to:193.9, dur:6.837, size:10544]
flow_2.79-20[ at:187.079 ,to:193.901, dur:6.822, size:5120]
flow_2.79-22[ at:187.095 ,to:193.901, dur:6.806, size:15000]
flow_2.79-24[ at:187.112 ,to:193.905, dur:6.793, size:8560]
flow_2.79-26[ at:187.129 ,to:193.907, dur:6.778, size:13808]
flow_2.79-28[ at:187.145 ,to:193.907, dur:6.762, size:15336]
flow_2.79-30[ at:187.161 ,to:193.907, dur:6.746, size:11728]
flow_2.79-32[ at:187.177 ,to:193.91, dur

In [26]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


186.916 [3416, 0, 3416, 0]
186.933 [13112, 0, 13112, 0]
186.95 [8112, 0, 8112, 0]
186.966 [15856, 0, 15856, 0]
186.983 [14936, 0, 14936, 0]
186.999 [7912, 0, 7912, 0]
187.015 [10248, 0, 10248, 0]
187.031 [10792, 0, 10792, 0]
187.047 [928, 0, 928, 0]
187.063 [10544, 0, 10544, 0]
187.079 [5120, 0, 5120, 0]
187.095 [15000, 0, 15000, 0]
187.112 [8560, 0, 8560, 0]
187.129 [13808, 0, 13808, 0]
187.145 [15336, 0, 15336, 0]
187.161 [11728, 0, 11728, 0]
187.177 [10688, 0, 10688, 0]
187.193 [16184, 0, 16184, 0]
187.209 [10824, 0, 10824, 0]
187.226 [4552, 0, 4552, 0]
187.243 [10104, 0, 10104, 0]
187.259 [11896, 0, 11896, 0]
187.276 [13368, 0, 13368, 0]
187.292 [6784, 0, 6784, 0]
187.308 [15592, 0, 15592, 0]
187.324 [15424, 0, 15424, 0]
187.34 [6464, 0, 6464, 0]
187.357 [16224, 0, 16224, 0]
187.373 [15480, 0, 15480, 0]
187.389 [14184, 0, 14184, 0]
187.405 [592, 0, 592, 0]
187.421 [8168, 0, 8168, 0]
187.438 [7816, 0, 7816, 0]
187.453 [2456, 0, 2456, 0]
187.469 [5736, 0, 5736, 0]
187.485 [8064, 0, 8

In [27]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])


186.916 [3416, 3416] 0
186.917 [3416, 3416] 0
186.918 [3416, 3416] 0
186.919 [3416, 3416] 0
186.92 [3416, 3416] 0
186.921 [3416, 3416] 0
186.922 [3416, 3416] 0
186.923 [3416, 3416] 0
186.924 [3416, 3416] 0
186.925 [3416, 3416] 0
186.926 [3416, 3416] 0
186.927 [3416, 3416] 0
186.928 [3416, 3416] 0
186.929 [3416, 3416] 0
186.93 [3416, 3416] 0
186.931 [3416, 3416] 0
186.932 [3416, 3416] 0
186.933 [16528, 16528] 0
186.934 [16528, 16528] 0
186.935 [16528, 16528] 0
186.936 [16528, 16528] 0
186.937 [16528, 16528] 0
186.938 [16528, 16528] 0
186.939 [16528, 16528] 0
186.94 [16528, 16528] 0
186.941 [16528, 16528] 0
186.942 [16528, 16528] 0
186.943 [16528, 16528] 0
186.944 [16528, 16528] 0
186.945 [16528, 16528] 0
186.946 [16528, 16528] 0
186.947 [16528, 16528] 0
186.948 [16528, 16528] 0
186.949 [16528, 16528] 0
186.95 [24640, 24640] 0
186.951 [24640, 24640] 0
186.952 [24640, 24640] 0
186.953 [24640, 24640] 0
186.954 [24640, 24640] 0
186.955 [24640, 24640] 0
186.956 [24640, 24640] 0
186.957 [2464